# First file for learning Jupyter
This is the first local file that I made to learn how to work iwth jupyter. 

In [ ]:
from csv import reader
# open files
google_store_link, apple_store_link = (open('raw_data/googleplaystore.csv', encoding="utf8"),
                                       open('raw_data/AppleStore.csv', encoding="utf8"))
# read the opened files
google_store, apple_store = (reader(google_store_link), 
                             reader(apple_store_link))
# convert the opened files into lists that are useable
apps_g, apps_a = (list(google_store), 
                  list(apple_store))
# extract data
apps_google, apps_apple = (apps_g[1:], 
                           apps_a[1:])
# extract header
header_google, header_apple = (apps_g[0], 
                               apps_a[0])


# Write an explore function

And glance at the data set to ensure the data is parsed as it is supposed to be. 


In [ ]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        
explore_data(apps_google, 0, 3, True)
print("\n")
explore_data(apps_apple, 0, 3, True)

# Data cleaning

## Objectives

1. Only retain apps that are free
2. Remove non-english apps


In [ ]:
print(apps_google[10471:10474])

Printing of the row 10472 along with a row above abd below indicates that there is an extra entry in the list at the end. Next, we remove it, instead of deleting the entire line. 

In [ ]:
if len(apps_google[10472]) == 13:
    del apps_google[10472]

In [ ]:
print(apps_google[10471:10474])

# Removed duplicate app entries

If we examine the rows main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show the data was collected at different times. We ce can use this information to build a criterion for removing the duplicates. The higher the number of reviews, the more recent the data should be. Rather than removing duplicates randomly, we'll only keep the row with the highest number of reviews and remove the other entries for any given a
pp.

In [ ]:
duplicate_apps = []
unique_apps = []

for row in apps_google:
    name = row[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])

In [ ]:
print("The header of the google dataset is: ", header_google)
print("The header of the apple dataset is: ", header_apple)

Create a dictionary that has all the unique app names as the keys and the ratings as the values associated with those keys. 

This dictionary will help us remove the duplicate data in next cell. 

In [ ]:
reviews_max = {}

for row in apps_google:
    name = row[0]
    n_reviews = float(row[3])

    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews
print(len(reviews_max))

        

Next cell we select only the rows that contain the data values tha match tha values in the dictionary. 

In [ ]:
google_clean = []
already_added = []

for row in apps_google:
    name = row[0]
    n_reviews = float(row[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        google_clean.append(row)
        already_added.append(name)
explore_data(google_clean, 0, 3, True)

# Removed non-English apps

The following code cell makes a function to test if the app name has at least 3 instances of  ASCII characters less than  or equal to 127. Everything else is non typical character. 

In [ ]:
def acs_127(input_string):
    counter = 0
    for character in input_string:
        if ord(character) > 127:
            counter += 1
    if counter > 3:
        return False
    else:
            return True

In [ ]:
print(acs_127(apps_apple[814][1]))

print(acs_127('Instagram'))
print(acs_127('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(acs_127('Docs To Go™ Free Office Suite'))
print(acs_127('Instachat 😜'))

In [ ]:
apps_apple_eng = []
apps_google_eng = []

for row in apps_apple:
    app_name = row[0]
    if acs_127(app_name):
        apps_apple_eng.append(row)

for row in google_clean:
    app_name = row[0]
    if acs_127(app_name):
        apps_google_eng.append(row)

explore_data(apps_apple_eng, 0, 3, True)

explore_data(apps_google_eng, 0, 3, True)

# Filter free apps

In [ ]:
apps_apple_eng_free = []
apps_google_eng_free = []

for row in apps_apple_eng:
    price = row[4]
    if price == "0":
        apps_apple_eng_free.append(row)

for row in apps_google_eng:
    price = row[7]
    if price == "0":
        apps_google_eng_free.append(row)

explore_data(apps_apple_eng_free, 0, 0, True)

explore_data(apps_google_eng_free, 0, 0, True)

# Most common apps by genre
Our goal is to determine the kinds of apps that are likely to attract more users because the number of people using our apps affect our revenue.

To minimize risks and overhead, our validation strategy for an app idea has three steps:1. 

Build a minimal Android version of the app, and add it to Google Pl2. ay.
If the app has a good response from users, we develop it furt3. her.
If the app is profitable after six months, we build an iOS version of the app and add it to the App 

tore.
Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful in both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

In [ ]:
def freq_table(df, index):
    table = {}
    total = 0
    
    for row in df:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages


apple_app_category = freq_table(apps_apple_eng_free, 11)
google_app_category = freq_table(apps_google_eng_free, 1)

print(apple_app_category)
print(google_app_category)

In [ ]:
def display_table(df, index):
    table = freq_table(df, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

display_table(apps_apple_eng_free, 11)
print("\n")
display_table(apps_google_eng_free, 1)
print("\n")
display_table(apps_google_eng_free, -4)

# Most Popular Apps by Genre on the App Store


In [ ]:
for genre in apple_app_category:
    total = 0 
    len_genre = 0
    for row in apps_apple_eng_free:
        genre_app = row[11]
        if genre_app == genre:
            user_rating = row[5]
            total += float(user_rating)
            len_genre += 1
    average_user_rating = total / len_genre
    print(genre, total, " : ", average_user_rating)

Profile recommendation will require us to sort this data. WIll tackle later. 

In [ ]:
print(header_google)


In [ ]:
# print(display_table(apps_google_eng_free, 1))
google_category_installs = {}
for category in google_app_category:
    total = 0
    len_category = 0
    
    for row in apps_google_eng_free:
        category_app = row[1]
        if category_app == category:
            installs = row[5]
            installs = installs.replace("+", "")
            installs = installs.replace(",", "")
            installs = float(installs)
            total += installs
            len_category += 1

    avg_installs = total / len_category
    google_category_installs[category] = avg_installs
print(google_category_installs)
           